In [302]:
import pandas as pd
import numpy as np
import dspy
from dotenv import load_dotenv
import ast
import torch

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer
from typing import Any


In [303]:
def get_label_list(dataset: Dataset):
    label_set = set()
    for data in dataset:
        labels = data["ner_tags"]
        label_set.update(labels)
    return list(label_set)

In [304]:
dataset_name = "darrow-ai/LegalLensNER-SharedTask"

In [305]:
from datasets import load_dataset
dataset = load_dataset(dataset_name)

In [306]:
def safe_literal_eval(value):
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # Return None
        return None

In [307]:
dataset = dataset.map(
    lambda x: {
        "tokens": safe_literal_eval(x["tokens"]),
        "ner_tags": safe_literal_eval(x["ner_tags"]),
    }
)

In [308]:
def is_not_none(example):
    # Check that 'tokens' and 'ner_tags' fields are not None
    return example["tokens"] is not None and example["ner_tags"] is not None

# Filter the dataset to remove examples with None values in 'tokens' or 'ner_tags'
dataset = dataset.filter(is_not_none)

In [309]:
def non_matching_len(example):
    # Check that the len of tokens matches the len of NER tags
    return len(example["tokens"]) == len(example["ner_tags"])

dataset = dataset.filter(non_matching_len)

In [310]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 970
    })
})

In [311]:
dataset = dataset["train"].train_test_split(test_size=0.3, seed=1234)

In [336]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 679
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 291
    })
})

In [312]:
label_list = get_label_list(
    dataset["train"]
)  # Assuming 'train' split exists and contains the labels

In [313]:
label_list

['B-VIOLATED BY',
 'O',
 'I-VIOLATION',
 'I-VIOLATED BY',
 'B-LAW',
 'I-VIOLATED ON',
 'I-LAW',
 'B-VIOLATED ON',
 'B-VIOLATION']

In [314]:
label_to_id = {label: i for i, label in enumerate(label_list)}
id_to_label = {i: label for i, label in enumerate(label_list)}

In [315]:
test_df = pd.read_excel("NER_test_set.xlsx")
test_set = Dataset.from_pandas(test_df)

In [316]:
test_set

Dataset({
    features: ['id', 'tokens'],
    num_rows: 380
})

In [317]:
import evaluate


metric = evaluate.load("seqeval")

In [318]:
def tokenize_and_align_labels(examples, tokenizer):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True,
    )
    labels = []
    for i, example_labels in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        last_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            elif word_id != last_word_id:
                label_id = label_to_id.get(example_labels[word_id], -100)
                label_ids.append(label_id)
            else:
                label_ids.append(label_id)
            last_word_id = word_id
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [319]:
def tokenize_dataset(model_checkpoint: str, dataset: Dataset):
    
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True, use_fast = True)
    
    # Tokenization and alignment of labels
    tokenized_dataset = dataset.map(
        lambda x: tokenize_and_align_labels(x, tokenizer), batched=True
    )
    
    return tokenizer, tokenized_dataset

In [320]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results


In [321]:
def train_model(model_checkpoint: str, tokenized_dataset: Any, label_list: list):
    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint, num_labels=len(label_list), id2label = id_to_label, label2id = label_to_id
    )
    
    training_args = TrainingArguments(model_checkpoint,
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=10,
                         warmup_steps=500,
                         weight_decay=0.01)
    
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    
    return trainer
    

In [322]:
def evaluate_trained_model(trainer: Trainer):
    eval_result = trainer.evaluate()
    
    evaluation_results = {k: v for k, v in eval_result.items()}
    
    return evaluation_results

In [323]:
# Function to run inference on a single example
def predict_tags(example, finetuned_tokenizer, finetuned_model):
    tokens = ast.literal_eval(example["tokens"])

    # Tokenize the input tokens
    inputs = finetuned_tokenizer(tokens, is_split_into_words=True, return_tensors="pt", truncation=True, padding=True)

    # Run inference
    with torch.no_grad():
        outputs = finetuned_model(**inputs)
        logits = outputs.logits

    # Convert logits to predicted tags
    predictions = torch.argmax(logits, dim=-1)
    predicted_tags = [finetuned_model.config.id2label[p.item()] for p in predictions[0]]

    # Align the predicted tags with the input tokens
    word_ids = inputs.word_ids()
    aligned_labels = []
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            continue
        if word_id != word_ids[i - 1]:
            aligned_labels.append(predicted_tags[i])

    # Return the predicted tags
    example["predicted_tags"] = aligned_labels
    return example

## Distilbert Cased

In [199]:
model_checkpoint = "distilbert/distilbert-base-uncased"

In [200]:
tokenizer, tokenized_dataset = tokenize_dataset(model_checkpoint, dataset)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 291/291 [00:00<00:00, 3468.83 examples/s]


In [201]:
trained_model = train_model(model_checkpoint, tokenized_dataset, label_list)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 10%|█         | 85/850 [00:40<05:40,  2.25it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 85/850 [00:45<05:

{'eval_loss': 1.09114670753479, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.7714780767194493, 'eval_runtime': 5.593, 'eval_samples_per_second': 52.029, 'eval_steps_per_second': 6.615, 'epoch': 1.0}


 20%|██        | 170/850 [01:26<05:03,  2.24it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 20%|██        | 170/850 [01:32<05:03,  2.24it/s]

{'eval_loss': 0.399384081363678, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.16806722689075632, 'eval_VIOLATION_recall': 0.19753086419753085, 'eval_VIOLATION_f1': 0.1816118047673099, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.16806722689075632, 'eval_overall_recall': 0.13029315960912052, 'eval_overall_f1': 0.14678899082568808, 'eval_overall_accuracy': 0.8886749771381817, 'eval_runtime': 5.5505, 'eval_samples_per_second': 52.428, 'eval_steps_per_second': 6.666, 'epoch': 2.0}


 30%|███       | 255/850 [02:12<04:23,  2.25it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 30%|███       | 255/850 [02:18<04:23,  2.25it/s]

...

{'eval_loss': 0.24343740940093994, 'eval_LAW_precision': 0.7835051546391752, 'eval_LAW_recall': 0.7676767676767676, 'eval_LAW_f1': 0.7755102040816326, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.5740740740740741, 'eval_VIOLATED BY_recall': 0.543859649122807, 'eval_VIOLATED BY_f1': 0.5585585585585585, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.35294117647058826, 'eval_VIOLATED ON_recall': 0.33962264150943394, 'eval_VIOLATED ON_f1': 0.34615384615384615, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.5823045267489712, 'eval_VIOLATION_recall': 0.6987654320987654, 'eval_VIOLATION_f1': 0.6352413019079685, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.5930232558139535, 'eval_overall_recall': 0.6644951140065146, 'eval_overall_f1': 0.6267281105990783, 'eval_overall_accuracy': 0.9441690330654089, 'eval_runtime': 5.6302, 'eval_samples_per_second': 51.685, 'eval_steps_per_second': 6.572, 'epoch': 9.0}


100%|██████████| 850/850 [07:43<00:00,  2.25it/s]

{'eval_loss': 0.2476310133934021, 'eval_LAW_precision': 0.7894736842105263, 'eval_LAW_recall': 0.7575757575757576, 'eval_LAW_f1': 0.7731958762886598, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.5614035087719298, 'eval_VIOLATED BY_recall': 0.5614035087719298, 'eval_VIOLATED BY_f1': 0.5614035087719298, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.37735849056603776, 'eval_VIOLATED ON_recall': 0.37735849056603776, 'eval_VIOLATED ON_f1': 0.3773584905660377, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.5533199195171026, 'eval_VIOLATION_recall': 0.6790123456790124, 'eval_VIOLATION_f1': 0.6097560975609756, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.5726495726495726, 'eval_overall_recall': 0.6547231270358306, 'eval_overall_f1': 0.6109422492401215, 'eval_overall_accuracy': 0.9436877316263176, 'eval_runtime': 5.5198, 'eval_samples_per_second': 52.72, 'eval_steps_per_second': 6.703, 'epoch': 10.0}


100%|██████████| 850/850 [07:44<00:00,  1.83it/s]

{'train_runtime': 464.6205, 'train_samples_per_second': 14.614, 'train_steps_per_second': 1.829, 'train_loss': 0.36275078044218173, 'epoch': 10.0}


In [202]:
evaluation_results = evaluate_trained_model(trained_model)

100%|██████████| 37/37 [00:05<00:00,  6.76it/s]


In [203]:
evaluation_results

{'eval_loss': 0.2476310133934021,
 'eval_LAW_precision': 0.7894736842105263,
 'eval_LAW_recall': 0.7575757575757576,
 'eval_LAW_f1': 0.7731958762886598,
 'eval_LAW_number': 99,
 'eval_VIOLATED BY_precision': 0.5614035087719298,
 'eval_VIOLATED BY_recall': 0.5614035087719298,
 'eval_VIOLATED BY_f1': 0.5614035087719298,
 'eval_VIOLATED BY_number': 57,
 'eval_VIOLATED ON_precision': 0.37735849056603776,
 'eval_VIOLATED ON_recall': 0.37735849056603776,
 'eval_VIOLATED ON_f1': 0.3773584905660377,
 'eval_VIOLATED ON_number': 53,
 'eval_VIOLATION_precision': 0.5533199195171026,
 'eval_VIOLATION_recall': 0.6790123456790124,
 'eval_VIOLATION_f1': 0.6097560975609756,
 'eval_VIOLATION_number': 405,
 'eval_overall_precision': 0.5726495726495726,
 'eval_overall_recall': 0.6547231270358306,
 'eval_overall_f1': 0.6109422492401215,
 'eval_overall_accuracy': 0.9436877316263176,
 'eval_runtime': 6.0683,
 'eval_samples_per_second': 47.954,
 'eval_steps_per_second': 6.097,
 'epoch': 10.0}

In [204]:
trained_model.save_model(output_dir="legal_lens_finetuned_models/distilbert_finetuned/")

In [205]:
tokenizer.save_pretrained(save_directory="legal_lens_finetuned_tokenizers/distilbert_finetuned/")

('legal_lens_finetuned_tokenizers/distilbert_finetuned/tokenizer_config.json',
 'legal_lens_finetuned_tokenizers/distilbert_finetuned/special_tokens_map.json',
 'legal_lens_finetuned_tokenizers/distilbert_finetuned/vocab.txt',
 'legal_lens_finetuned_tokenizers/distilbert_finetuned/added_tokens.json',
 'legal_lens_finetuned_tokenizers/distilbert_finetuned/tokenizer.json')

In [227]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("legal_lens_finetuned_tokenizers/distilbert_finetuned/")
finetuned_model = AutoModelForTokenClassification.from_pretrained("legal_lens_finetuned_models/distilbert_finetuned/")

In [230]:
# Apply the function to the entire dataset
test_results = test_set.map(
    lambda x: predict_tags(x, finetuned_tokenizer, finetuned_model))

Map: 100%|██████████| 380/380 [00:10<00:00, 37.98 examples/s]


In [231]:
test_results

Dataset({
    features: ['id', 'tokens', 'predicted_tags'],
    num_rows: 380
})

## BERT Uncased

In [232]:
model_checkpoint = "google-bert/bert-base-uncased"

In [210]:
tokenizer, tokenized_dataset = tokenize_dataset(model_checkpoint, dataset)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 291/291 [00:00<00:00, 3849.82 examples/s]


In [211]:
trained_model = train_model(model_checkpoint, tokenized_dataset, label_list)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

{'eval_loss': 0.8390417695045471, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.7714780767194493, 'eval_runtime': 9.306, 'eval_samples_per_second': 31.27, 'eval_steps_per_second': 3.976, 'epoch': 1.0}


 20%|██        | 170/850 [02:35<09:16,  1.22it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 20%|██        | 170/850 [02:44<09:16,  1.22it/s]

{'eval_loss': 0.38071727752685547, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.1673228346456693, 'eval_VIOLATION_recall': 0.20987654320987653, 'eval_VIOLATION_f1': 0.18619934282584885, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.1673228346456693, 'eval_overall_recall': 0.13843648208469056, 'eval_overall_f1': 0.15151515151515155, 'eval_overall_accuracy': 0.8921884776435481, 'eval_runtime': 9.2588, 'eval_samples_per_second': 31.43, 'eval_steps_per_second': 3.996, 'epoch': 2.0}


 30%|███       | 255/850 [04:00<08:07,  1.22it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 30%|███       | 255/850 [04:09<08:07,  1.22it/s]

...

{'eval_loss': 0.25872042775154114, 'eval_LAW_precision': 0.7604166666666666, 'eval_LAW_recall': 0.7373737373737373, 'eval_LAW_f1': 0.7487179487179487, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.6071428571428571, 'eval_VIOLATED BY_recall': 0.5964912280701754, 'eval_VIOLATED BY_f1': 0.6017699115044247, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.4107142857142857, 'eval_VIOLATED ON_recall': 0.4339622641509434, 'eval_VIOLATED ON_f1': 0.42201834862385323, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.5494949494949495, 'eval_VIOLATION_recall': 0.671604938271605, 'eval_VIOLATION_f1': 0.6044444444444445, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.5718349928876245, 'eval_overall_recall': 0.6547231270358306, 'eval_overall_f1': 0.6104783599088839, 'eval_overall_accuracy': 0.9439765124897723, 'eval_runtime': 9.2728, 'eval_samples_per_second': 31.382, 'eval_steps_per_second': 3.99, 'epoch': 9.0}


100%|██████████| 850/850 [13:59<00:00,  1.22it/s]

{'eval_loss': 0.2607121467590332, 'eval_LAW_precision': 0.7708333333333334, 'eval_LAW_recall': 0.7474747474747475, 'eval_LAW_f1': 0.758974358974359, 'eval_LAW_number': 99, 'eval_VIOLATED BY_precision': 0.5666666666666667, 'eval_VIOLATED BY_recall': 0.5964912280701754, 'eval_VIOLATED BY_f1': 0.5811965811965812, 'eval_VIOLATED BY_number': 57, 'eval_VIOLATED ON_precision': 0.38596491228070173, 'eval_VIOLATED ON_recall': 0.41509433962264153, 'eval_VIOLATED ON_f1': 0.4, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.5496957403651116, 'eval_VIOLATION_recall': 0.6691358024691358, 'eval_VIOLATION_f1': 0.6035634743875278, 'eval_VIOLATION_number': 405, 'eval_overall_precision': 0.5679886685552408, 'eval_overall_recall': 0.6530944625407166, 'eval_overall_f1': 0.6075757575757575, 'eval_overall_accuracy': 0.9437358617702267, 'eval_runtime': 9.2966, 'eval_samples_per_second': 31.302, 'eval_steps_per_second': 3.98, 'epoch': 10.0}


100%|██████████| 850/850 [14:00<00:00,  1.01it/s]

{'train_runtime': 840.7453, 'train_samples_per_second': 8.076, 'train_steps_per_second': 1.011, 'train_loss': 0.3335383437661564, 'epoch': 10.0}


In [212]:
evaluation_results = evaluate_trained_model(trained_model)

100%|██████████| 37/37 [00:09<00:00,  4.00it/s]


In [213]:
evaluation_results

{'eval_loss': 0.2607121467590332,
 'eval_LAW_precision': 0.7708333333333334,
 'eval_LAW_recall': 0.7474747474747475,
 'eval_LAW_f1': 0.758974358974359,
 'eval_LAW_number': 99,
 'eval_VIOLATED BY_precision': 0.5666666666666667,
 'eval_VIOLATED BY_recall': 0.5964912280701754,
 'eval_VIOLATED BY_f1': 0.5811965811965812,
 'eval_VIOLATED BY_number': 57,
 'eval_VIOLATED ON_precision': 0.38596491228070173,
 'eval_VIOLATED ON_recall': 0.41509433962264153,
 'eval_VIOLATED ON_f1': 0.4,
 'eval_VIOLATED ON_number': 53,
 'eval_VIOLATION_precision': 0.5496957403651116,
 'eval_VIOLATION_recall': 0.6691358024691358,
 'eval_VIOLATION_f1': 0.6035634743875278,
 'eval_VIOLATION_number': 405,
 'eval_overall_precision': 0.5679886685552408,
 'eval_overall_recall': 0.6530944625407166,
 'eval_overall_f1': 0.6075757575757575,
 'eval_overall_accuracy': 0.9437358617702267,
 'eval_runtime': 10.6067,
 'eval_samples_per_second': 27.436,
 'eval_steps_per_second': 3.488,
 'epoch': 10.0}

In [233]:
tokenizer.save_pretrained(save_directory="legal_lens_finetuned_tokenizers/bert_uncased_finetuned/")
trained_model.save_model(output_dir="legal_lens_finetuned_models/bert_uncased_finetuned/")

In [234]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("legal_lens_finetuned_tokenizers/bert_uncased_finetuned/")
finetuned_model = AutoModelForTokenClassification.from_pretrained("legal_lens_finetuned_models/bert_uncased_finetuned/")

In [235]:
# Apply the function to the entire dataset
test_results = test_set.map(
    lambda x: predict_tags(x, finetuned_tokenizer, finetuned_model))

Map: 100%|██████████| 380/380 [00:16<00:00, 22.80 examples/s]


## Bert Cased

In [236]:
model_checkpoint = "google-bert/bert-base-cased"

In [237]:
tokenizer, tokenized_dataset = tokenize_dataset(model_checkpoint, dataset)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 291/291 [00:00<00:00, 3991.70 examples/s]


In [238]:
trained_model = train_model(model_checkpoint, tokenized_dataset, label_list)

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

{'eval_loss': 0.8149062991142273, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 108, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 486, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.7665760105036106, 'eval_runtime': 9.2376, 'eval_samples_per_second': 31.502, 'eval_steps_per_second': 4.005, 'epoch': 1.0}


 20%|██        | 170/850 [02:36<09:14,  1.23it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 20%|██        | 170/850 [02:45<09:14,  1.23it/s]

{'eval_loss': 0.3455027639865875, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 108, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.2629695885509839, 'eval_VIOLATION_recall': 0.30246913580246915, 'eval_VIOLATION_f1': 0.2813397129186602, 'eval_VIOLATION_number': 486, 'eval_overall_precision': 0.2620320855614973, 'eval_overall_recall': 0.20704225352112676, 'eval_overall_f1': 0.2313139260424862, 'eval_overall_accuracy': 0.8946356560067523, 'eval_runtime': 9.2057, 'eval_samples_per_second': 31.611, 'eval_steps_per_second': 4.019, 'epoch': 2.0}


 30%|███       | 255/850 [04:09<08:04,  1.23it/s]

...

{'eval_loss': 0.2539927065372467, 'eval_LAW_precision': 0.8446601941747572, 'eval_LAW_recall': 0.8055555555555556, 'eval_LAW_f1': 0.8246445497630333, 'eval_LAW_number': 108, 'eval_VIOLATED BY_precision': 0.6052631578947368, 'eval_VIOLATED BY_recall': 0.7301587301587301, 'eval_VIOLATED BY_f1': 0.6618705035971223, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.4098360655737705, 'eval_VIOLATED ON_recall': 0.4716981132075472, 'eval_VIOLATED ON_f1': 0.43859649122807015, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.6440366972477064, 'eval_VIOLATION_recall': 0.7222222222222222, 'eval_VIOLATION_f1': 0.6808923375363725, 'eval_VIOLATION_number': 486, 'eval_overall_precision': 0.6484076433121019, 'eval_overall_recall': 0.7169014084507043, 'eval_overall_f1': 0.6809364548494984, 'eval_overall_accuracy': 0.9483259870580512, 'eval_runtime': 9.5379, 'eval_samples_per_second': 30.51, 'eval_steps_per_second': 3.879, 'epoch': 9.0}


100%|██████████| 850/850 [14:02<00:00,  1.20it/s]

{'eval_loss': 0.26457345485687256, 'eval_LAW_precision': 0.8165137614678899, 'eval_LAW_recall': 0.8240740740740741, 'eval_LAW_f1': 0.8202764976958524, 'eval_LAW_number': 108, 'eval_VIOLATED BY_precision': 0.6176470588235294, 'eval_VIOLATED BY_recall': 0.6666666666666666, 'eval_VIOLATED BY_f1': 0.6412213740458016, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.391304347826087, 'eval_VIOLATED ON_recall': 0.5094339622641509, 'eval_VIOLATED ON_f1': 0.44262295081967207, 'eval_VIOLATED ON_number': 53, 'eval_VIOLATION_precision': 0.6423487544483986, 'eval_VIOLATION_recall': 0.742798353909465, 'eval_VIOLATION_f1': 0.6889312977099238, 'eval_VIOLATION_number': 486, 'eval_overall_precision': 0.6423267326732673, 'eval_overall_recall': 0.7309859154929578, 'eval_overall_f1': 0.6837944664031621, 'eval_overall_accuracy': 0.9487948982462722, 'eval_runtime': 9.6788, 'eval_samples_per_second': 30.066, 'eval_steps_per_second': 3.823, 'epoch': 10.0}


100%|██████████| 850/850 [14:07<00:00,  1.00it/s]

{'train_runtime': 847.7884, 'train_samples_per_second': 8.009, 'train_steps_per_second': 1.003, 'train_loss': 0.3135357576258042, 'epoch': 10.0}


In [239]:
evaluation_results = evaluate_trained_model(trained_model)

100%|██████████| 37/37 [00:09<00:00,  3.93it/s]


In [240]:
evaluation_results

{'eval_loss': 0.26457345485687256,
 'eval_LAW_precision': 0.8165137614678899,
 'eval_LAW_recall': 0.8240740740740741,
 'eval_LAW_f1': 0.8202764976958524,
 'eval_LAW_number': 108,
 'eval_VIOLATED BY_precision': 0.6176470588235294,
 'eval_VIOLATED BY_recall': 0.6666666666666666,
 'eval_VIOLATED BY_f1': 0.6412213740458016,
 'eval_VIOLATED BY_number': 63,
 'eval_VIOLATED ON_precision': 0.391304347826087,
 'eval_VIOLATED ON_recall': 0.5094339622641509,
 'eval_VIOLATED ON_f1': 0.44262295081967207,
 'eval_VIOLATED ON_number': 53,
 'eval_VIOLATION_precision': 0.6423487544483986,
 'eval_VIOLATION_recall': 0.742798353909465,
 'eval_VIOLATION_f1': 0.6889312977099238,
 'eval_VIOLATION_number': 486,
 'eval_overall_precision': 0.6423267326732673,
 'eval_overall_recall': 0.7309859154929578,
 'eval_overall_f1': 0.6837944664031621,
 'eval_overall_accuracy': 0.9487948982462722,
 'eval_runtime': 9.8114,
 'eval_samples_per_second': 29.659,
 'eval_steps_per_second': 3.771,
 'epoch': 10.0}

In [241]:
trained_model.save_model(output_dir="legal_lens_finetuned_models/bert_cased_finetuned/")

In [242]:
tokenizer.save_pretrained(save_directory="legal_lens_finetuned_tokenizers/bert_cased_finetuned/")

('legal_lens_finetuned_tokenizers/bert_cased_finetuned/tokenizer_config.json',
 'legal_lens_finetuned_tokenizers/bert_cased_finetuned/special_tokens_map.json',
 'legal_lens_finetuned_tokenizers/bert_cased_finetuned/vocab.txt',
 'legal_lens_finetuned_tokenizers/bert_cased_finetuned/added_tokens.json',
 'legal_lens_finetuned_tokenizers/bert_cased_finetuned/tokenizer.json')

In [243]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("legal_lens_finetuned_tokenizers/bert_cased_finetuned/")
finetuned_model = AutoModelForTokenClassification.from_pretrained("legal_lens_finetuned_models/bert_cased_finetuned/")

In [244]:
# Apply the function to the entire dataset
test_results = test_set.map(
    lambda x: predict_tags(x, finetuned_tokenizer, finetuned_model))

Map: 100%|██████████| 380/380 [00:15<00:00, 24.49 examples/s]


## Roberta

In [324]:
model_checkpoint = "FacebookAI/roberta-base"

In [326]:
tokenizer, tokenized_dataset = tokenize_dataset(model_checkpoint, dataset)

/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|██████████| 291/291 [00:00<00:00, 4202.71 examples/s]


In [327]:
trained_model = train_model(model_checkpoint, tokenized_dataset, label_list)

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 10%|█         | 85/850 [01:14<10:33,  1.21it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 10%|█         | 85/850 [01:23<10:33,  1.21it/s]

{'eval_loss': 0.6782763600349426, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 98, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 58, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 51, 'eval_VIOLATION_precision': 0.0, 'eval_VIOLATION_recall': 0.0, 'eval_VIOLATION_f1': 0.0, 'eval_VIOLATION_number': 359, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.776010339513844, 'eval_runtime': 9.4571, 'eval_samples_per_second': 30.771, 'eval_steps_per_second': 3.912, 'epoch': 1.0}


 20%|██        | 170/850 [02:39<09:16,  1.22it/s]/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

 20%|██        | 170/850 [02:48<09:16,  1.22it/s]

{'eval_loss': 0.3421342670917511, 'eval_LAW_precision': 0.0, 'eval_LAW_recall': 0.0, 'eval_LAW_f1': 0.0, 'eval_LAW_number': 98, 'eval_VIOLATED BY_precision': 0.0, 'eval_VIOLATED BY_recall': 0.0, 'eval_VIOLATED BY_f1': 0.0, 'eval_VIOLATED BY_number': 58, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 51, 'eval_VIOLATION_precision': 0.2835820895522388, 'eval_VIOLATION_recall': 0.37047353760445684, 'eval_VIOLATION_f1': 0.32125603864734303, 'eval_VIOLATION_number': 359, 'eval_overall_precision': 0.2835820895522388, 'eval_overall_recall': 0.23498233215547704, 'eval_overall_f1': 0.25700483091787435, 'eval_overall_accuracy': 0.8944176567082567, 'eval_runtime': 9.3778, 'eval_samples_per_second': 31.031, 'eval_steps_per_second': 3.946, 'epoch': 2.0}


 30%|███       | 255/850 [04:11<08:08,  1.22it/s]

...

{'eval_loss': 0.21662026643753052, 'eval_LAW_precision': 0.8173076923076923, 'eval_LAW_recall': 0.8673469387755102, 'eval_LAW_f1': 0.8415841584158416, 'eval_LAW_number': 98, 'eval_VIOLATED BY_precision': 0.6142857142857143, 'eval_VIOLATED BY_recall': 0.7413793103448276, 'eval_VIOLATED BY_f1': 0.671875, 'eval_VIOLATED BY_number': 58, 'eval_VIOLATED ON_precision': 0.40298507462686567, 'eval_VIOLATED ON_recall': 0.5294117647058824, 'eval_VIOLATED ON_f1': 0.45762711864406785, 'eval_VIOLATED ON_number': 51, 'eval_VIOLATION_precision': 0.5905882352941176, 'eval_VIOLATION_recall': 0.6991643454038997, 'eval_VIOLATION_f1': 0.6403061224489797, 'eval_VIOLATION_number': 359, 'eval_overall_precision': 0.6096096096096096, 'eval_overall_recall': 0.7173144876325088, 'eval_overall_f1': 0.6590909090909092, 'eval_overall_accuracy': 0.9495451608092658, 'eval_runtime': 9.0622, 'eval_samples_per_second': 32.111, 'eval_steps_per_second': 4.083, 'epoch': 9.0}


100%|██████████| 850/850 [14:01<00:00,  1.22it/s]

{'eval_loss': 0.2228051722049713, 'eval_LAW_precision': 0.8817204301075269, 'eval_LAW_recall': 0.8367346938775511, 'eval_LAW_f1': 0.8586387434554974, 'eval_LAW_number': 98, 'eval_VIOLATED BY_precision': 0.7096774193548387, 'eval_VIOLATED BY_recall': 0.7586206896551724, 'eval_VIOLATED BY_f1': 0.7333333333333333, 'eval_VIOLATED BY_number': 58, 'eval_VIOLATED ON_precision': 0.6, 'eval_VIOLATED ON_recall': 0.5882352941176471, 'eval_VIOLATED ON_f1': 0.5940594059405941, 'eval_VIOLATED ON_number': 51, 'eval_VIOLATION_precision': 0.6190476190476191, 'eval_VIOLATION_recall': 0.724233983286908, 'eval_VIOLATION_f1': 0.6675224646983312, 'eval_VIOLATION_number': 359, 'eval_overall_precision': 0.6656, 'eval_overall_recall': 0.734982332155477, 'eval_overall_f1': 0.6985726280436608, 'eval_overall_accuracy': 0.9533230600984243, 'eval_runtime': 9.1178, 'eval_samples_per_second': 31.916, 'eval_steps_per_second': 4.058, 'epoch': 10.0}


100%|██████████| 850/850 [14:02<00:00,  1.01it/s]

{'train_runtime': 842.4538, 'train_samples_per_second': 8.06, 'train_steps_per_second': 1.009, 'train_loss': 0.2861541938781738, 'epoch': 10.0}


In [328]:
evaluation_results = evaluate_trained_model(trained_model)

100%|██████████| 37/37 [00:09<00:00,  4.01it/s]


In [329]:
evaluation_results

{'eval_loss': 0.2228051722049713,
 'eval_LAW_precision': 0.8817204301075269,
 'eval_LAW_recall': 0.8367346938775511,
 'eval_LAW_f1': 0.8586387434554974,
 'eval_LAW_number': 98,
 'eval_VIOLATED BY_precision': 0.7096774193548387,
 'eval_VIOLATED BY_recall': 0.7586206896551724,
 'eval_VIOLATED BY_f1': 0.7333333333333333,
 'eval_VIOLATED BY_number': 58,
 'eval_VIOLATED ON_precision': 0.6,
 'eval_VIOLATED ON_recall': 0.5882352941176471,
 'eval_VIOLATED ON_f1': 0.5940594059405941,
 'eval_VIOLATED ON_number': 51,
 'eval_VIOLATION_precision': 0.6190476190476191,
 'eval_VIOLATION_recall': 0.724233983286908,
 'eval_VIOLATION_f1': 0.6675224646983312,
 'eval_VIOLATION_number': 359,
 'eval_overall_precision': 0.6656,
 'eval_overall_recall': 0.734982332155477,
 'eval_overall_f1': 0.6985726280436608,
 'eval_overall_accuracy': 0.9533230600984243,
 'eval_runtime': 9.6556,
 'eval_samples_per_second': 30.138,
 'eval_steps_per_second': 3.832,
 'epoch': 10.0}

In [330]:
trained_model.save_model(output_dir="legal_lens_finetuned_models/roberta_finetuned/")

In [331]:
tokenizer.save_pretrained(save_directory="legal_lens_finetuned_tokenizers/roberta_finetuned/")

('legal_lens_finetuned_tokenizers/roberta_finetuned/tokenizer_config.json',
 'legal_lens_finetuned_tokenizers/roberta_finetuned/special_tokens_map.json',
 'legal_lens_finetuned_tokenizers/roberta_finetuned/vocab.json',
 'legal_lens_finetuned_tokenizers/roberta_finetuned/merges.txt',
 'legal_lens_finetuned_tokenizers/roberta_finetuned/added_tokens.json',
 'legal_lens_finetuned_tokenizers/roberta_finetuned/tokenizer.json')

In [332]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("legal_lens_finetuned_tokenizers/roberta_finetuned/")
finetuned_model = AutoModelForTokenClassification.from_pretrained("legal_lens_finetuned_models/roberta_finetuned")

In [333]:
# Apply the function to the entire dataset
test_results = test_set.map(
    lambda x: predict_tags(x, finetuned_tokenizer, finetuned_model))

Map: 100%|██████████| 380/380 [00:15<00:00, 24.96 examples/s]


In [335]:
test_results.to_csv("NER_test_set_results_roberta.csv")

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 26.09ba/s]


492164

## FLAN T5

In [259]:
model_checkpoint = "google/flan-t5-base"

In [260]:
tokenizer, tokenized_dataset = tokenize_dataset(model_checkpoint, dataset)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|█████████

In [261]:
trained_model = train_model(model_checkpoint, tokenized_dataset, label_list)

Some weights of T5ForTokenClassification were not initialized from the model checkpoint at google/flan-t5-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/khalidrajan/anaconda3/envs/llm_dev_python3.8/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 10%|█         | 85/850 [01:47<13:31,  1.06s/it]

{'eval_loss': 3.8418359756469727, 'eval_LAW_precision': 0.00019428793471925395, 'eval_LAW_recall': 0.008928571428571428, 'eval_LAW_f1': 0.00038030043734550294, 'eval_LAW_number': 112, 'eval_VIOLATED BY_precision': 0.000686655985351339, 'eval_VIOLATED BY_recall': 0.047619047619047616, 'eval_VIOLATED BY_f1': 0.0013537906137184117, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.0005629045876723895, 'eval_VIOLATED ON_recall': 0.038461538461538464, 'eval_VIOLATED ON_f1': 0.0011095700416088763, 'eval_VIOLATED ON_number': 52, 'eval_VIOLATION_precision': 0.008405438813349814, 'eval_VIOLATION_recall': 0.07538802660753881, 'eval_VIOLATION_f1': 0.015124555160142347, 'eval_VIOLATION_number': 451, 'eval_overall_precision': 0.002337267734018932, 'eval_overall_recall': 0.058997050147492625, 'eval_overall_f1': 0.004496402877697842, 'eval_overall_accuracy': 0.1591245680434425, 'eval_runtime': 13.3459, 'eval_samples_per_second': 21.804, 'eval_steps_per_second': 2.772, 'epoch': 1.0}


 20%|██        | 170/850 [03:34<11:48,  1.04s/it]

{'eval_loss': 1.3803106546401978, 'eval_LAW_precision': 0.0006993006993006993, 'eval_LAW_recall': 0.008928571428571428, 'eval_LAW_f1': 0.001297016861219196, 'eval_LAW_number': 112, 'eval_VIOLATED BY_precision': 0.00338409475465313, 'eval_VIOLATED BY_recall': 0.031746031746031744, 'eval_VIOLATED BY_f1': 0.006116207951070336, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.0, 'eval_VIOLATED ON_recall': 0.0, 'eval_VIOLATED ON_f1': 0.0, 'eval_VIOLATED ON_number': 52, 'eval_VIOLATION_precision': 0.009124087591240875, 'eval_VIOLATION_recall': 0.04434589800443459, 'eval_VIOLATION_f1': 0.01513431706394249, 'eval_VIOLATION_number': 451, 'eval_overall_precision': 0.004878048780487805, 'eval_overall_recall': 0.03392330383480826, 'eval_overall_f1': 0.008529575375486742, 'eval_overall_accuracy': 0.6776369919368109, 'eval_runtime': 13.3517, 'eval_samples_per_second': 21.795, 'eval_steps_per_second': 2.771, 'epoch': 2.0}


 30%|███       | 255/850 [05:22<10:19,  1.04s/it]

...

{'eval_loss': 0.273849755525589, 'eval_LAW_precision': 0.4854368932038835, 'eval_LAW_recall': 0.44642857142857145, 'eval_LAW_f1': 0.4651162790697675, 'eval_LAW_number': 112, 'eval_VIOLATED BY_precision': 0.08108108108108109, 'eval_VIOLATED BY_recall': 0.047619047619047616, 'eval_VIOLATED BY_f1': 0.06, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.05263157894736842, 'eval_VIOLATED ON_recall': 0.038461538461538464, 'eval_VIOLATED ON_f1': 0.044444444444444446, 'eval_VIOLATED ON_number': 52, 'eval_VIOLATION_precision': 0.22163588390501318, 'eval_VIOLATION_recall': 0.37250554323725055, 'eval_VIOLATION_f1': 0.27791563275434245, 'eval_VIOLATION_number': 451, 'eval_overall_precision': 0.23824786324786323, 'eval_overall_recall': 0.32890855457227136, 'eval_overall_f1': 0.27633209417596033, 'eval_overall_accuracy': 0.9319565575119302, 'eval_runtime': 12.9778, 'eval_samples_per_second': 22.423, 'eval_steps_per_second': 2.851, 'epoch': 9.0}


100%|██████████| 850/850 [17:56<00:00,  1.18s/it]

{'eval_loss': 0.2719593346118927, 'eval_LAW_precision': 0.49514563106796117, 'eval_LAW_recall': 0.45535714285714285, 'eval_LAW_f1': 0.47441860465116287, 'eval_LAW_number': 112, 'eval_VIOLATED BY_precision': 0.05555555555555555, 'eval_VIOLATED BY_recall': 0.031746031746031744, 'eval_VIOLATED BY_f1': 0.04040404040404041, 'eval_VIOLATED BY_number': 63, 'eval_VIOLATED ON_precision': 0.02564102564102564, 'eval_VIOLATED ON_recall': 0.019230769230769232, 'eval_VIOLATED ON_f1': 0.02197802197802198, 'eval_VIOLATED ON_number': 52, 'eval_VIOLATION_precision': 0.22576361221779548, 'eval_VIOLATION_recall': 0.376940133037694, 'eval_VIOLATION_f1': 0.28239202657807305, 'eval_VIOLATION_number': 451, 'eval_overall_precision': 0.24060150375939848, 'eval_overall_recall': 0.3303834808259587, 'eval_overall_f1': 0.27843380981976384, 'eval_overall_accuracy': 0.9323679447095606, 'eval_runtime': 15.0725, 'eval_samples_per_second': 19.307, 'eval_steps_per_second': 2.455, 'epoch': 10.0}


100%|██████████| 850/850 [17:57<00:00,  1.27s/it]

{'train_runtime': 1077.5198, 'train_samples_per_second': 6.302, 'train_steps_per_second': 0.789, 'train_loss': 1.3680954248764936, 'epoch': 10.0}


In [262]:
evaluation_results = evaluate_trained_model(trained_model)

100%|██████████| 37/37 [00:13<00:00,  2.74it/s]


In [263]:
evaluation_results

{'eval_loss': 0.2719593346118927,
 'eval_LAW_precision': 0.49514563106796117,
 'eval_LAW_recall': 0.45535714285714285,
 'eval_LAW_f1': 0.47441860465116287,
 'eval_LAW_number': 112,
 'eval_VIOLATED BY_precision': 0.05555555555555555,
 'eval_VIOLATED BY_recall': 0.031746031746031744,
 'eval_VIOLATED BY_f1': 0.04040404040404041,
 'eval_VIOLATED BY_number': 63,
 'eval_VIOLATED ON_precision': 0.02564102564102564,
 'eval_VIOLATED ON_recall': 0.019230769230769232,
 'eval_VIOLATED ON_f1': 0.02197802197802198,
 'eval_VIOLATED ON_number': 52,
 'eval_VIOLATION_precision': 0.22576361221779548,
 'eval_VIOLATION_recall': 0.376940133037694,
 'eval_VIOLATION_f1': 0.28239202657807305,
 'eval_VIOLATION_number': 451,
 'eval_overall_precision': 0.24060150375939848,
 'eval_overall_recall': 0.3303834808259587,
 'eval_overall_f1': 0.27843380981976384,
 'eval_overall_accuracy': 0.9323679447095606,
 'eval_runtime': 15.355,
 'eval_samples_per_second': 18.952,
 'eval_steps_per_second': 2.41,
 'epoch': 10.0}

In [270]:
trained_model.save_model(output_dir="legal_lens_finetuned_models/flan_t5_base_finetuned/")

In [271]:
tokenizer.save_pretrained(save_directory="legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/")

('legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/tokenizer_config.json',
 'legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/special_tokens_map.json',
 'legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/spiece.model',
 'legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/added_tokens.json',
 'legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/tokenizer.json')

In [272]:
finetuned_tokenizer = AutoTokenizer.from_pretrained("legal_lens_finetuned_tokenizers/flan_t5_base_finetuned/")
finetuned_model = AutoModelForTokenClassification.from_pretrained("legal_lens_finetuned_models/flan_t5_base_finetuned/")

In [273]:
# Apply the function to the entire dataset
test_results = test_set.map(
    lambda x: predict_tags(x, finetuned_tokenizer, finetuned_model))

Map: 100%|██████████| 380/380 [00:21<00:00, 17.70 examples/s]
